# Lab4: KKT Conditions 

University of California Berkeley

ME C231A, EE C220B, Experiential Advanced Control I

***

*This* lab introduces how to check the Karush-Kuhn-Tucker (KKT) conditions using $\texttt{Pyomo}$.

In [1]:
# This cell is commented out because Pyomo and solvers are already installed on Docker image.
# Please run this cell only if you are using Google Colab. 

# # install required dependencies
# import sys
# IN_COLAB = 'google.colab' in sys.modules
# if IN_COLAB:
#   !pip install -q pyomo
#   !apt-get install -y -qq glpk-utils
#   !apt-get install -y -qq coinor-cbc
#   !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#   !unzip -o -q ipopt-linux64

***

# Quadratic Programming Example
For the following problems write a quadratic program and solve using $\texttt{Pyomo}$. Then, check whether the KKT conditions are satisfied or not. 

\begin{align}
\min_{z_1,z_2}~ &  z_1^2 + z_2^2 \\
& z_1 \geq 1  \\
& z_2 \geq 1 
\end{align}

<!-- <img src='KKT_slide.png' width="700"/>  -->

In [2]:
import numpy as np
import pyomo.environ as pyo
from pyomo.opt import SolverStatus, TerminationCondition
threshold = 1e-5

model = pyo.ConcreteModel()


model.z1 = pyo.Var()
model.z2 = pyo.Var()

model.Obj = pyo.Objective(expr = model.z1**2 + model.z2**2)
model.Constraint1 = pyo.Constraint(expr = -model.z1 + 1 <= 0)
model.Constraint2 = pyo.Constraint(expr = -model.z2 + 1 <= 0)
# model.Constraint2 = pyo.Constraint(expr = 1 <= model.z2) # Do not write the constraitns in non-standard form, which can result in getting dual variables with different signs. 

model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

print('dual 1:', model.dual[model.Constraint1])
print('dual 2:', model.dual[model.Constraint2])
        
print('z1*_solver =', pyo.value(model.z1))
print('z2*_solver =', pyo.value(model.z2))

print('opt_value =', pyo.value(model.Obj))

dual 1: -1.9999999825076669
dual 2: -1.9999999825076669
z1*_solver = 0.9999999912538334
z2*_solver = 0.9999999912538334
opt_value = 1.999999965015334


In [4]:
if results.solver.termination_condition != TerminationCondition.optimal:
    KKTsat = False
else:
    A = -np.eye(2)
    b = -np.ones((2,))
    zOpt = np.array([pyo.value(model.z1), pyo.value(model.z2)])
    
    u = []
    for c in model.component_objects(pyo.Constraint, active=True):
        print ("Constraint", c)
        for index in c:
            u.append(-model.dual[c[index]]) # The duals in pyomo are defined for positive inequality constraints g(x) > =0, so we add a negative sign.
            print(model.dual[c[index]])
    u = np.asarray(u)
    for i in range(len(u)):
        if (u[i] < threshold) & (u[i] > -threshold):
            u[i] = 0 
            
# Checking KKT Conditions: 

    flag_primal = np.any(np.all(A@zOpt <= b + threshold) | np.all(A@zOpt <= b - threshold))   #  A@zOpt <= b primal feasibility 
    
    flag_dual = np.all(u >= 0)     # dual feasibility 
    
    flag_cs = np.all(np.multiply(u,(A@zOpt-b)) < threshold) & np.all(np.multiply(u,(A@zOpt-b)) > -threshold)  # complementary slackness u1*g1 = 0, u2*g2=0, where g1 and g2 are the two inequality constraints. 
    
    grad_lagrangian = [2*zOpt[0],2*zOpt[1]] + u.T@A
    
    for i in range(len(grad_lagrangian)):
        if (grad_lagrangian[i] < threshold) & (grad_lagrangian[i] > -threshold):  # gradient of Lagragian evaluated at optimizer point must be zero.    
            grad_lagrangian[i] = 0
    flag_grad = np.all(grad_lagrangian == 0)
    KKT_conditions = np.array([flag_primal, flag_dual, flag_cs, flag_grad])
    if all(KKT_conditions == 1):
        KKTsat = True
    else:
        KKTsat = False   

print(KKTsat)

Constraint Constraint1
-1.9999999825076669
Constraint Constraint2
-1.9999999825076669
True
